In [1]:
# Load libraries
import pandas as pd
import numpy as np
import geopy.distance
import os
import math
from datetime import timedelta, datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

In [2]:
# Read the biotic and accompanying acoustic files
filename = "Biotic_06SL794.csv"
acoustic_1 = pd.read_csv("Data\\Acoustic\\Acoustic" + filename[6:99], header=0)
biotic_1 = pd.read_csv("Data\\Biotic\\" + filename, header=0)

# Filter out the required columns
a_columns = ['LogTime', 'LogLatitude', 'LogLongitude']
b_columns = ['HaulStartTime', 'HaulDuration', 'HaulValidity', 'HaulStartLatitude', 'HaulStartLongitude', 'HaulStopLatitude', 'HaulStopLongitude', 'HaulDistance', 'HaulTowDirection']
acoustic_1_1 = acoustic_1[a_columns]
biotic_1_1 = biotic_1[b_columns]

# Set the datetime columns to correct datatype and drop duplicates
acoustic_1_1['LogTime'] = pd.to_datetime(acoustic_1_1['LogTime'])
biotic_1_1['HaulStartTime'] = pd.to_datetime(biotic_1_1['HaulStartTime'])
acoustic_1_1 = acoustic_1_1.drop_duplicates()

# Prepare the biotic file to have the same structure as the acoustic file
bio_to_aco = biotic_1_1[['HaulStartTime', 'HaulStartLatitude', 'HaulStartLongitude']]
bio_to_aco = bio_to_aco.rename(columns={'HaulStartTime': 'LogTime', 'HaulStartLatitude': 'LogLatitude', 'HaulStartLongitude': 'LogLongitude'})

# Concat the biotic and acoustic file to create a df containing all registered timestamps
acoustic_final = pd.concat([acoustic_1_1, bio_to_aco], ignore_index=True)
acoustic_final = acoustic_final.sort_values('LogTime', ignore_index=True)
acoustic_final['fileIDN'] = 1

# Index both final sets over their respective timestamp columns
acoustic_final2 = acoustic_final.set_index('LogTime')
biotic_final = biotic_1_1.set_index('HaulStartTime')

# Perform final concat adding all available columns to the available timestamps and create new column containing binary classifier for fishing or not.
full = pd.concat([acoustic_final2, biotic_final], axis=1)
full['fishing'] = 0
full.loc[full['HaulValidity'] == 'V', 'fishing'] = 1
full = full.drop('HaulValidity', axis=1)

# Extend the fishing period (fishing = 1) over all rows that are within the start-end haul timeframe
# -----------------------------------------------------------------------------------------------------------------------------
time_str = '1/1/1900 12:00:00.0000'
date_format_str = '%d/%m/%Y %H:%M:%S.%f'

track = 1
start_haul = datetime.strptime(time_str, date_format_str)
end_haul = datetime.strptime(time_str, date_format_str)
index = full.index
while track < full.shape[0]:
        if full['HaulDuration'][track] > 0:
                start_haul = index[track]
                end_haul = index[track] + timedelta(minutes=full['HaulDuration'][track])

        if index[track] >= start_haul and index[track] <= end_haul:
                full['fishing'][track] = 1
        track += 1
# -----------------------------------------------------------------------------------------------------------------------------


# Calculate speed
# -----------------------------------------------------------------------------------------------------------------------------
full = full[['LogLatitude', 'LogLongitude', 'fishing', 'fileIDN', ]]
distances = [0]
durations = [0]
index = full.index
i=0
while i < full.shape[0] - 1:
        distances.append(geopy.distance.distance((full['LogLatitude'][i], full['LogLongitude'][i]), (full['LogLatitude'][i+1], full['LogLongitude'][i+1])).km)
        durations.append((index[i+1] - index[i]).total_seconds())
        i += 1

km_s = [0]
j=1
while j < len(distances):
        if durations[j] != 0:
                km_s.append((distances[j]/durations[j]) * 60 * 60)
        else:
                km_s.append(0)
        j += 1

full['speed'] = km_s

# speed_lag_1 = []
# speed_lead_1 = []
# speed_both_1 = []
# speed_lag_2 = []
# speed_lead_2 = []
# speed_both_2 = []

# full['lag_1'] = full['speed'].shift(1)
# full['lag_2'] = full['speed'].shift(2)
# full['lead_1'] = full['speed'].shift(-1)
# full['lead_2'] = full['speed'].shift(-2)

# m = 0
# while m < full.shape[0]:
#         speed_current = full['speed'][m]
#         lag_1 = full['lag_1'][m]
#         lag_2 = full['lag_2'][m]
#         lead_1 = full['lead_1'][m]
#         lead_2 = full['lead_2'][m]

#         average_speed_lg1 = (speed_current + lag_1) / 2
#         average_speed_lg2 = (speed_current + lag_1 + lag_2) / 3
#         if np.isnan(lead_1):
#                 average_speed_ld1 = speed_current
#         else:
#                 average_speed_ld1 = (speed_current + lead_1) / 2
#         average_speed_ld2 = (speed_current + lead_1 + lead_2) / 3
#         average_speed_b1 = (speed_current + lead_1 + lag_1) / 3
#         average_speed_b2 = (speed_current + lead_1 + lag_1 + lead_2 + lag_2) / 5

#         speed_lag_1.append(average_speed_lg1)
#         speed_lag_2.append(average_speed_lg2)
#         speed_lead_1.append(average_speed_ld1)
#         speed_lead_2.append(average_speed_ld2)
#         speed_both_1.append(average_speed_b1)
#         speed_both_2.append(average_speed_b2)
        
#         m += 1

# # print(speed_lag_1)

# full['speed_lag_1'] = speed_lag_1
# full['speed_lag_2'] = speed_lag_2
# full['speed_lead_1'] = speed_lead_1
# full['speed_lead_2'] = speed_lead_2
# full['speed_both_1'] = speed_both_1
# full['speed_both_2'] = speed_both_2


# Calculate direction based on previous and next
# -----------------------------------------------------------------------------------------------------------------------------
# Create lag and lead of lat/long
full['lat_f1'] = full['LogLatitude'].shift(-1) # latitude with lead 1
full['lon_f1'] = full['LogLongitude'].shift(-1) # Longtitude with lead 1
full['lat_b1'] = full['LogLatitude'].shift(1) # latitude with lag 1
full['lon_b1'] = full['LogLongitude'].shift(1) # Longtitude with lag 1

direction = [0]
geo_track = 1
while geo_track < full.shape[0]:
        delta_lon = full['LogLongitude'][geo_track] - full['lon_b1'][geo_track]
        y2 = math.sin(delta_lon) * math.cos(full['LogLatitude'][geo_track])
        x2 = math.cos(full['lat_b1'][geo_track]) * math.sin(full['LogLatitude'][geo_track]) - math.sin(full['lat_b1'][geo_track]) * math.cos(full['LogLatitude'][geo_track]) * math.cos(delta_lon)

        brng2 = math.atan2(y2, x2)
        brng2 = math.degrees(brng2)

        direction.append(brng2)

        geo_track += 1

full['bearing'] = direction

direction_change = [0]
geo_track2 = 1
while geo_track2 < full.shape[0]:
        change = full['bearing'][geo_track2 - 1] - full['bearing'][geo_track2]
        direction_change.append(change)

        geo_track2 += 1

full['bearing change'] = direction_change
# -----------------------------------------------------------------------------------------------------------------------------

C:\Users\MarkR\AppData\Local\Temp\ipykernel_12580\384415671.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  acoustic_1_1['LogTime'] = pd.to_datetime(acoustic_1_1['LogTime'])
C:\Users\MarkR\AppData\Local\Temp\ipykernel_12580\384415671.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  biotic_1_1['HaulStartTime'] = pd.to_datetime(biotic_1_1['HaulStartTime'])
C:\Users\MarkR\AppData\Local\Temp\ipykernel_12580\384415671.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(full)

In [3]:
filename = "Biotic_06SL794.csv"
acoustic_1 = pd.read_csv("Data\\Acoustic\\Acoustic" + filename[6:99], header=0)
biotic_1 = pd.read_csv("Data\\Biotic\\" + filename, header=0)

In [4]:
display(acoustic_1)

,Data,Header,LogDistance,LogTime,LogLatitude,LogLongitude,LogOrigin,LogLatitude2,LogLongitude2,LogOrigin2,...,CalibrationID,DataAcquisitionID,DataProcessingID,SamplePingAxisIntervalOrigin,DataSaCategory,EchoTypeID,DataType,DataUnit,DataValue,CruiseLocalID
0,Data,Record,1,2021-07-01 04:23:00,55.895245,6.030530,start,NaN,NaN,NaN,...,CAL1,DAQ1,DPR1,NaN,CLU,NaN,C,m2nmi-2,0.0,06SL794
1,Data,Record,1,2021-07-01 04:23:00,55.895245,6.030530,start,NaN,NaN,NaN,...,CAL1,DAQ1,DPR1,NaN,CLU,NaN,C,m2nmi-2,0.0,06SL794
2,Data,Record,1,2021-07-01 04:23:00,55.895245,6.030530,start,NaN,NaN,NaN,...,CAL1,DAQ1,DPR1,NaN,CLU,NaN,C,m2nmi-2,0.0,06SL794
3,Data,Record,1,2021-07-01 04:23:00,55.895245,6.030530,start,NaN,NaN,NaN,...,CAL1,DAQ1,DPR1,NaN,CLU,NaN,C,m2nmi-2,0.0,06SL794
4,Data,Record,1,2021-07-01 04:23:00,55.895245,6.030530,start,NaN,NaN,NaN,...,CAL1,DAQ1,DPR1,NaN,CLU,NaN,C,m2nmi-2,0.0,06SL794
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91207,Data,Record,2874,2021-07-19 15:56:00,54.786270,5.968703,start,NaN,NaN,NaN,...,CAL1,DAQ1,DPR1,NaN,MIX,NaN,C,m2nmi-2,0.0,06SL794
91208,Data,Record,2874,2021-07-19 15:56:00,54.786270,5.968703,start,NaN,NaN,NaN,...,CAL1,DAQ1,DPR1,NaN,UNK,NaN,C,m2nmi-2,0.0,06SL794
91209,Data,Record,2874,2021-07-19 15:56:00,54.786270,5.968703,start,NaN,NaN,NaN,...,CAL1,DAQ1,DPR1,NaN,CLU,NaN,C,m2nmi-2,0.0,06SL794
91210,Data,Record,2874,2021-07-19 15:56:00,54.786270,5.968703,start,NaN,NaN,NaN,...,CAL1,DAQ1,DPR1,NaN,MIX,NaN,C,m2nmi-2,0.0,06SL794


In [5]:
display(biotic_1)

,Haul,Header,CruiseLocalID,HaulGear,HaulNumber,HaulStationName,HaulStartTime,HaulDuration,HaulValidity,HaulStartLatitude,...,HaulHydrographicStationID,HaulTowDirection,HaulSpeedGround,HaulSpeedWater,HaulWindDirection,HaulWindSpeed,HaulSwellDirection,HaulSwellHeight,HaulLogDistance,HaulStratum
0,Haul,Record,06SL794,PEL,1,4,2021-07-01T08:59,7,V,55.89600,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Haul,Record,06SL794,PEL,2,7,2021-07-01T14:56,30,V,55.59350,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Haul,Record,06SL794,PEL,3,9,2021-07-02T06:32,31,V,55.59533,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Haul,Record,06SL794,PEL,4,12,2021-07-02T10:54,15,V,55.29367,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Haul,Record,06SL794,PEL,5,15,2021-07-03T04:33,8,V,55.29467,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Haul,Record,06SL794,PEL,6,17,2021-07-03T07:21,20,V,54.99533,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Haul,Record,06SL794,PEL,7,18,2021-07-03T10:03,33,V,54.99500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Haul,Record,06SL794,PEL,8,19,2021-07-03T13:59,15,V,54.99450,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Haul,Record,06SL794,PEL,9,22,2021-07-04T05:42,30,V,54.69517,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Haul,Record,06SL794,PEL,10,23,2021-07-04T08:11,30,V,54.69550,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
